# Exercise

Modify this notebook and apply the following classifiers for the Product Review Problem - Classify Product Reviews as Positive or Not:

1) MLPclassifier (https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html)

2) Naive Bayes (https://scikit-learn.org/stable/modules/naive_bayes.html) 


Write a comparison report of the above classifiers and knn


# K Nearest Neighbors Model for a Classification Problem: Classify Product Reviews as Positive or Negative

In this notebook, we use the K Nearest Neighbors method to build a classifier to predict the __isPositive__ field of our review dataset (that is very similar to the final project dataset).


1. <a href="#1">Reading the dataset</a>
2. <a href="#2">Exploratory data analysis</a>
3. <a href="#3">Text Processing: Stop words removal and stemming</a>
4. <a href="#4">Train - Validation Split</a>
5. <a href="#5">Using the Naive Baye</a>
6. <a href="#6">Using the MLPclassifier</a>



## 1. <a name="1">Reading the dataset</a>
(<a href="#0">Go to top</a>)

We will use the __pandas__ library to read our dataset.

In [2]:
import pandas as pd

df = pd.read_csv('./data/imdb_train.csv')

print('The shape of the dataset is:', df.shape)

The shape of the dataset is: (25000, 2)


Let's look at the first 10 rows of the dataset. 

In [3]:
df.head(10)

,text,label
0,This movie makes me want to throw up every tim...,0
1,Listening to the director's commentary confirm...,0
2,One of the best Tarzan films is also one of it...,1
3,Valentine is now one of my favorite slasher fi...,1
4,No mention if Ann Rivers Siddons adapted the m...,0
5,Several years ago the Navy kept a studied dist...,1
6,This is a masterpiece footage in B/W 35mm film...,1
7,Such a long awaited movie.. But it has disappo...,0
8,When two writers make a screenplay of a horror...,1
9,"Make no mistake, Maureen O'Sullivan is easily ...",1


## 2. <a name="2">Exploratory data analysis</a>
(<a href="#0">Go to top</a>)

Let's look at the distribution of __isPositive__ field.

In [5]:
df["label"].value_counts()

label
0    12500
1    12500
Name: count, dtype: int64

We can check the number of missing values for each columm below.

In [6]:
print(df.isna().sum())

text     0
label    0
dtype: int64


We have missing values in our text fields.

## 3. <a name="3">Text Processing: Stop words removal and stemming</a>
(<a href="#0">Go to top</a>)

In [7]:
# Install the library and functions
import nltk

nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ptolo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ptolo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

We will create the stop word removal and text cleaning processes below. NLTK library provides a list of common stop words. We will use the list, but remove some of the words from that list (because those words are actually useful to understand the sentiment in the sentence).

In [8]:
import nltk, re
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from nltk.tokenize import word_tokenize

# Let's get a list of stop words from the NLTK library
stop = stopwords.words('english')

# These words are important for our problem. We don't want to remove them.
excluding = ['against', 'not', 'don', "don't",'ain', 'aren', "aren't", 'couldn', "couldn't",
             'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 
             'haven', "haven't", 'isn', "isn't", 'mightn', "mightn't", 'mustn', "mustn't",
             'needn', "needn't",'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', 
             "weren't", 'won', "won't", 'wouldn', "wouldn't"]

# New stop word list
stop_words = [word for word in stop if word not in excluding]

snow = SnowballStemmer('english')

def process_text(texts): 
    final_text_list=[]
    for sent in texts:
        
        # Check if the sentence is a missing value
        if isinstance(sent, str) == False:
            sent = ""
            
        filtered_sentence=[]
        
        sent = sent.lower() # Lowercase 
        sent = sent.strip() # Remove leading/trailing whitespace
        sent = re.sub('\s+', ' ', sent) # Remove extra space and tabs
        sent = re.compile('<.*?>').sub('', sent) # Remove HTML tags/markups:
        
        for w in word_tokenize(sent):
            # We are applying some custom filtering here, feel free to try different things
            # Check if it is not numeric and its length>2 and not in stop words
            if(not w.isnumeric()) and (len(w)>2) and (w not in stop_words):  
                # Stem and add to filtered list
                filtered_sentence.append(snow.stem(w))
        final_string = " ".join(filtered_sentence) #final string of cleaned words
 
        final_text_list.append(final_string)
        
    return final_text_list

## 4. <a name="4">Train - Validation Split</a>
(<a href="#0">Go to top</a>)

Let's split our dataset into training (90%) and validation (10%). 

In [28]:
from sklearn.model_selection import train_test_split


X_train, X_val, y_train, y_val = train_test_split(df[["text"]],
                                                  df["label"],
                                                  test_size=0.10,
                                                  shuffle=True,
                                                  random_state=324
                                                 )

In [30]:
print("Processing the reviewText fields")
train_text_list = process_text(X_train["text"].tolist())
val_text_list = process_text(X_val["text"].tolist())

Processing the reviewText fields


Our __process_text()__ method in section 3 uses empty string for missing values.

## 5. <a name="5">Using the Naive Baye</a>
(<a href="#0">Go to top</a>)


In [32]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import BernoulliNB

### PIPELINE ###
##########################

pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True,
                                  max_features=15)),
    ('BernoulliNB',  BernoulliNB())  
                                ])

# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=15)),
                ('BernoulliNB', BernoulliNB())])

In [35]:
# We using lists of processed text fields 
X_train = train_text_list
X_val = val_text_list

# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=15)),
                ('BernoulliNB', BernoulliNB())])

In [36]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[721 559]
 [498 722]]
              precision    recall  f1-score   support

           0       0.59      0.56      0.58      1280
           1       0.56      0.59      0.58      1220

    accuracy                           0.58      2500
   macro avg       0.58      0.58      0.58      2500
weighted avg       0.58      0.58      0.58      2500

Accuracy (validation): 0.5772


## 6. <a name="6">Using the MLPclassifier</a>
(<a href="#0">Go to top</a>)

In [37]:
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer

### PIPELINE ###
##########################

pipeline = Pipeline([
    ('text_vect', CountVectorizer(binary=True,
                                  max_features=15)),
    ('MLPClassifier', MLPClassifier())  
                                ])

# Visualize the pipeline
# This will come in handy especially when building more complex pipelines, stringing together multiple preprocessing steps
from sklearn import set_config
set_config(display='diagram')
pipeline

Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=15)),
                ('MLPClassifier', MLPClassifier())])

In [38]:
# Fit the Pipeline to training data
pipeline.fit(X_train, y_train.values)

d:\app\anaconda3\envs\ml\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:686: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


Pipeline(steps=[('text_vect', CountVectorizer(binary=True, max_features=15)),
                ('MLPClassifier', MLPClassifier())])

In [39]:
# Use the fitted pipeline to make predictions on the validation dataset
val_predictions = pipeline.predict(X_val)
print(confusion_matrix(y_val.values, val_predictions))
print(classification_report(y_val.values, val_predictions))
print("Accuracy (validation):", accuracy_score(y_val.values, val_predictions))

[[732 548]
 [549 671]]
              precision    recall  f1-score   support

           0       0.57      0.57      0.57      1280
           1       0.55      0.55      0.55      1220

    accuracy                           0.56      2500
   macro avg       0.56      0.56      0.56      2500
weighted avg       0.56      0.56      0.56      2500

Accuracy (validation): 0.5612
